In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from matplotlib import pyplot as plt

In [5]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)

In [6]:
device

device(type='cuda')

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [8]:
transform

Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)

In [12]:
trainset = torchvision.datasets.CIFAR10(
    "data",
    train=True,
    download=True,
    transform=transform
)

100%|██████████| 170M/170M [00:37<00:00, 4.51MB/s] 


In [13]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [14]:
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=64,
    shuffle=True,
    num_workers=2
)

In [15]:
test_set = torchvision.datasets.CIFAR10(
    "data",
    train=False,
    download=True,
    transform=transform
)

In [16]:
testloader = torch.utils.data.DataLoader(
    test_set,
    batch_size=64,
    shuffle=False,
    num_workers=2
) 

In [17]:
# Class names
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [18]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3,
                               32,
                               kernel_size=3,
                               padding=1)
        
        self.conv2 = nn.Conv2d(32,
                               64,
                               kernel_size=3,
                               padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2,
                                 stride=2)
        
        self.fc1 = nn.Linear(64 * 8 * 8, 128)

        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [19]:
model = SimpleCNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader):.4f}")


Epoch 1, Loss: 1.3374
Epoch 2, Loss: 0.9683
Epoch 3, Loss: 0.8209
Epoch 4, Loss: 0.7144
Epoch 5, Loss: 0.6151
Epoch 6, Loss: 0.5251
Epoch 7, Loss: 0.4393
Epoch 8, Loss: 0.3615
Epoch 9, Loss: 0.2907
Epoch 10, Loss: 0.2325


In [21]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 71.08%
